# Introducción al Aprendizaje Profundo

<center>
    <img src="figures/intro-deep-1.jpg" width="1300"/>
</center>

## Generalidades

Una ANN realiza **composiciones de funciones** simples para formar una función más compleja.

En teoría, una única composición de funciones simples puede aproximar a casi cualquier función compleja, pero esto requiere una **enorme cantidad de parámetros**.

<center>
    <img src="figures/ann-ex-1.png" width="400"/>
</center>

Incluso es posible demostrar que un MLP con varias capas ocultas usando la **función identidad**, no tiene ninguna ventaja frente al MLP con solo una capa oculta.


El aprendizaje profundo realiza **repetidas composiciones** de funciones no-lineales (muchas capas ocultas), lo que tiene un gran poder expresivo

\begin{align}
    \hat{\mathbf{Y}} = f_1( f_2( f_3( ... f_n( \mathbf{X} ) ) ) )
\end{align}

Esto puede reducir considerablemente la cantidad de parámetros totales necesarios dependiendo de las funciones de activación usadas.

<center>
    <img src="figures/intro-deep-2.jpg" width="600"/>
</center>



Las arquitecturas profundas aprovechan mejor los **patrones repetitivos** en los datos para lograr mejor generalización incluso en áreas del espacio con pocos o sin datos.

Usualmente estos patrones repetitivos se aprenden como pesos de **atributos jerarquizados**.

<center>
    <img src="figures/intro-deep-3.gif" width="800"/>
</center>



## Ejemplo
Una función unidimensional toma alternadamente valores +1 o -1 un total de 8 veces. ¿De qué forma puede construirse una ANN para aproximar esta función?

<center>
    <img src="figures/funscale-deep-1.png" width="600"/>
</center>

Usando una **ANN superficial**, deberían por lo menos haber 8 unidades para aproximar cada valor (sin contar las unidades de sesgo u offset)

Así, dependiendo de la entrada, debería "activarse" una de las 8 unidades para entregar la salida correcta

<center>
    <img src="figures/ann-ex-3.png" width="1000"/>
</center>

Usando una **ANN profunda**, se podrían usar 3 capas de 2 unidades para obtener un total de 8 "caminos" que se activan para entregar la salida correcta dependiendo de la entrada.

<center>
    <img src="figures/funscale-deep-3.png" width="500"/>
</center>

Así, la ANN profunda aprende parámetros de forma **jerarquizada**. 

Por ejemplo, la primera capa aprende un escalón simple, la segunda capa aprende un escalón doble, y sucesivamente.

In [11]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

X = [[0.], [1.], [2.], [3.], [4.], [5.], [6.], [7.]]
y = [1., -1., 1., -1., 1., -1., 1., -1.]

clf = MLPRegressor(random_state=4, tol=1.e-8, alpha=1.e-8, hidden_layer_sizes=(128,),
                   activation='tanh', solver='lbfgs', max_iter=int(1e8), batch_size=1)
clf.fit(X, y)
y_predict = clf.predict(X)
print(mean_squared_error(y, y_predict))
print(y_predict)

4.328055637578201e-10
[ 1.00000689 -1.00000619  0.99999465 -1.00000652  0.99999522 -0.99997812
  0.99996138 -0.99996377]


In [14]:
clf = MLPRegressor(random_state=4, tol=1.e-8, alpha=1.e-8, hidden_layer_sizes=16*(8, ),
                   activation='tanh', solver='lbfgs', max_iter=int(1e8), batch_size=1)
clf.fit(X, y)
y_predict = clf.predict(X)
print(mean_squared_error(y, y_predict))
print(y_predict)

0.37675587489828755
[ 1.00079232 -0.99999358  0.99944706 -0.98411517  0.82330869 -0.20941274
 -0.35643113 -0.28052778]


# Arquitecturas comunes

## Redes superficiales (shallow network)
La mayoría de los modelos de aprendizaje automático (regresión lineal, logística, SVM, PCA, etc.) pueden simularse como una red neuronal de 1 ó 2 capas ocultas.

<center>
    <img src="figures/ann-5.png" width="500"/>
</center>

Un tipo de red superficial sería el perceptrón multicapa con 1 ó 2 capas ocultas.

Estas redes están totalmente conectadas, es decir, hay conexión directa entre todas las unidades.

Son un tipo de red prealimentada (feed forward) en las que el flujo va direccionalmente desde capas anteriores a capas posteriores.

<center>
    <img src="figures/ann-gif-1.gif" width="700"/>
</center>

## Redes de base radial (RBF, radial basis function)

A pesar de no ser profunda, difiere de las redes prealimentadas ya que tienen una parte entrenada no supervisadamente y una parte entrenada supervisadamente.

Comúnmente tienen una capa oculta con una cantidad de unidades superior al de la capa de entrada.

<center>
    <img src="figures/rbf-1.png" width="700"/>
</center>

Primero, de manera no supervisada se obtiene el **vector prototipo** $\bar{\mu_i}$ para la $i$-ésima unidad oculta. Además, se obtiene un **ancho de banda** $\sigma_i$ para cada unidad oculta.

Luego, para cada vector de atributos $\bar{X}$ que pasa a la capa oculta se define la **función de activación radial** $\phi_{i}(\bar{X})$ como sigue:

\begin{split}
    h_{i} = \phi_{i}(\bar{X}) = \text{exp} \left( {- \frac{ \| \bar{X} - \bar{\mu_i} \|^{2} }{ 2 \sigma_{i}^2 } } \right) \; \forall i \in \{1, ..., m\}
\end{split}

<center>
    <img src="figures/rbf-2.jpeg" width="500"/>
</center>

Cada una de las $m$ unidades ocultas tendrá gran influencia en los datos cercanos a su vector prototipo. Es decir, este tipo de red actúa como un método de **agrupamiento**.

Luego se calcula la predicción en la capa de salida de la red como se fuera un **perceptrón**, es decir:

\begin{split}
    \hat{y} = \sum_{i=1}^{m} w_{i} \phi_{i}(\bar{X}) = \sum_{i=1}^{m} w_{i} \text{exp} \left( {- \frac{ \| \bar{X} - \bar{\mu_i} \|^{2} }{ 2 \sigma_{i}^2 } } \right)
\end{split}

Los valores de los pesos $w_{i}$ se aprenden de forma **supervisada** como en una red prealimentada. También se implementa una neurona de sesgo.

## Máquinas restringidas de Boltzmann

Utilizan el concepto de minimización de energía para crear una red neuronal de forma no supervisada.

También se puede ser un entrenamiento posterior de forma supervisada.

Es diferente a las redes prealimentadas en varios sentidos:
- Modela la **probabilidad conjunta** de los atributos en vez de minimizar una función de costo.
- Tienen **flujo no directo**, ya que aprenden relaciones probabilísticas en vez de mapeos entrada-salida.
- Crea representaciones **latentes** (ocultas) de los datos.

## Redes neuronales recurrentes

Las conexiones entre unidades pueden crear **ciclos**, por lo que la salida de una neurona puede afectar a su propia entrada.

Son especialmente útiles para modelar comportamiento **temporales** o dinámicos.

<center>
    <img src="figures/recurrent-1.png" width="500"/>
</center>


## Redes neuronales convolucionales 

Históricamente han sido el tipo de ANN más exitosa, usadas principalmente para reconocimiento de imágenes, localización de objetos y procesamiento de texto, entre otros.

Están inspiradas por el funcionamiento del cortex visual de los gatos, en donde porciones específicas del campo visual activan ciertas neuronas.

<center>
    <img src="figures/convolution-2-gif.gif" width="800"/>
</center>

## Redes generativas adversativas



<center>
    <img src="figures/GAN.png" width="700"/>
</center>

# Implementación en Python

## Librerías

Hay principalmente 3 librerías para implementar modelos de aprendizaje profundo en Python:

- Tensorflow
- Pytorch
- Keras

Estas librerías permiten formar fácilmente capas convolucionales, pooling, etc.

Se diferencian en cuanto a la complejidad de usar, compatibilidad, velocidad, etc.

<center>
    <img src="figures/comparison-deep.png" width="1200"/>
</center>

## Uso de la GPU

Debido a la considerable mayor cantidad de atributos y operaciones computacionales presentes en una red profunda, los tiempos de entrenamiento son mucho mayores.

Esto se logra solucionar mediante el uso de la tarjeta gráfica (GPU) en vez del procesador central (CPU) para realizar los cálculos.

<center>
    <img src="figures/gpu-1.png" width="800"/>
</center>

El poder usar la GPU para los cálculos se logra mediante la plataforma **CUDA** de Nvidia.

Las librerías anteriores pueden detectar si la GPU del PC es compatible con CUDA. 

<center>
    <img src="figures/cuda.jpg" width="500"/>
</center>

En **Keras**, si la GPU es compatible los cálculos se realizarán automáticamente en ella.

Aquí se puede revisar el listado de GPUs compatibles:

https://developer.nvidia.com/cuda-gpus



## Links para configuración de TensorFlow

https://www.tensorflow.org/install/pip?hl=es-419#windows-wsl2_1

https://www.tensorflow.org/install/gpu?hl=es-419

https://www.youtube.com/watch?v=0S81koZpwPA